# 레이어 선택 분석 노트북

이 노트북은 이미 수행된 실험 결과(`results/*_bert_qwen2_*_results.txt`)와
레이어 선택 로그(`results/layer_selection/.../selection.json`)를 이용해:

- 태스크·모델별 **최적(best) / 마지막(last) / 휴리스틱(auto) 레이어** 비교
- last 대비 best 성능 차이 중, 휴리스틱이 얼마나 회수하는지
- 전체 레이어–성능 곡선에서 선택된 레이어가 어디에 위치하는지

를 요약·시각화하기 위한, 논문용 분석 노트북입니다. 학습 코드는 포함하지 않고,
순수히 보고서/시각화를 위한 post-hoc 분석만 다룹니다.

In [1]:
from pathlib import Path
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 그림 스타일 설정 (논문용 기본 스타일)
sns.set(style="whitegrid")

RESULTS_ROOT = Path("results")

## 결과 및 로그 로딩 헬퍼 함수

이 섹션에서는:

- 레이어별 그리드 서치 결과(`*_bert_qwen2_*_results.txt`)를 DataFrame으로 변환하는 함수
- 레이어 선택 로그(`layer_selection/.../selection.json`)에서 선택 레이어를 읽어오는 함수
- 단일 태스크에 대해 best / last / selected 레이어 성능을 한 줄로 요약하는 함수

를 정의합니다.

In [2]:
def parse_grid_results(path: Path) -> pd.DataFrame:
    """results/{dataset}_bert_{llm_type}_results.txt 파일을 DataFrame으로 파싱."""
    rows = []
    if not path.exists():
        return pd.DataFrame(columns=["layer", "acc", "f1"])  # empty
    for line in path.read_text().splitlines():
        line = line.strip()
        if not line:
            continue
        parts = line.split()
        kv = {}
        for p in parts:
            if "=" in p:
                k, v = p.split("=", 1)
                kv[k] = v
        if "layer" not in kv or "acc" not in kv:
            continue
        layer = int(kv["layer"])
        acc = float(kv["acc"])
        f1 = float(kv.get("f1", kv.get("macro_f1", kv["acc"])))
        rows.append({"layer": layer, "acc": acc, "f1": f1})
    return pd.DataFrame(rows).sort_values("layer").reset_index(drop=True)


def load_selection_layer(task: str, dataset: str, llm_type: str) -> int | None:
    """results/layer_selection/.../selection.json 에서 best_llm_layer(또는 유사 키)를 읽어옴."""
    sel_path = RESULTS_ROOT / "layer_selection" / task / dataset / "bert" / llm_type / "selection.json"
    if not sel_path.exists():
        return None
    data = json.loads(sel_path.read_text())
    if "best_llm_layer" in data:
        return int(data["best_llm_layer"])
    for key in ("final_layer", "L_Apply", "L_Abstract"):
        if key in data:
            try:
                return int(data[key])
            except Exception:
                continue
    return None


def summarize_task(dataset: str, llm_type: str, task: str = "classification") -> pd.Series | None:
    """단일 (dataset, llm_type)에 대한 best / last / sel 요약 정보를 한 줄로 반환."""
    grid_path = RESULTS_ROOT / f"{dataset}_bert_{llm_type}_results.txt"
    df = parse_grid_results(grid_path)
    if df.empty:
        return None
    best = df.iloc[df["acc"].idxmax()]
    last = df.iloc[df["layer"].idxmax()]
    sel_layer = load_selection_layer(task, dataset, llm_type)
    if sel_layer is not None and sel_layer in df["layer"].values:
        sel = df[df["layer"] == sel_layer].iloc[0]
    else:
        sel = None
    out = {
        "dataset": dataset,
        "llm_type": llm_type,
        "best_layer": int(best["layer"]),
        "best_acc": best["acc"],
        "last_layer": int(last["layer"]),
        "last_acc": last["acc"],
    }
    if sel is not None:
        out.update(
            {
                "sel_layer": int(sel["layer"]),
                "sel_acc": sel["acc"],
                "delta_sel_last": sel["acc"] - last["acc"],
                "delta_sel_best": sel["acc"] - best["acc"],
            }
        )
    return pd.Series(out)


## Qwen2-1.5B 분류 태스크 요약 테이블

여기서는 Qwen2-1.5B + bert 조합에 대해 주요 분류 태스크별:

- 최적 레이어(best)
- 마지막 레이어(last)
- selection 로그에서 나온 레이어(sel)

의 정확도와, sel이 last/best와 비교해 얼마나 좋아/나쁜지를 표로 정리합니다.

In [3]:
datasets = ["sst2", "cola", "imdb", "tweet_offensive", "tweet_sentiment_binary"]
llm_type = "qwen2_1.5b"

rows = []
for ds in datasets:
    s = summarize_task(ds, llm_type, task="classification")
    if s is not None:
        rows.append(s)

summary_df = pd.DataFrame(rows)
summary_df

""


## 예시: SST-2의 레이어–성능 곡선 (Qwen2-1.5B)

SST-2에 대해 레이어 인덱스를 x축, test accuracy를 y축으로 두고,

- 최적 레이어(best)
- 마지막 레이어(last)
- selection에서 선택된 레이어(sel)

의 위치를 함께 시각화합니다. 이 그림은 논문 본문/부록에 "레이어별 성능 프로파일" 예시로 사용 가능합니다.

In [4]:
ds = "sst2"
grid = parse_grid_results(RESULTS_ROOT / f"{ds}_bert_{llm_type}_results.txt")
best = grid.iloc[grid["acc"].idxmax()]
last = grid.iloc[grid["layer"].idxmax()]
sel_layer = load_selection_layer("classification", ds, llm_type)
sel = grid[grid["layer"] == sel_layer].iloc[0] if sel_layer in grid["layer"].values else None

plt.figure(figsize=(6, 4))
plt.plot(grid["layer"], grid["acc"], marker="o", label="acc vs layer")
plt.scatter([best["layer"]], [best["acc"]], color="green", label="best", zorder=3)
plt.scatter([last["layer"]], [last["acc"]], color="red", label="last", zorder=3)
if sel is not None:
    plt.scatter([sel["layer"]], [sel["acc"]], color="orange", label="selected", zorder=3)
plt.xlabel("LLM layer index")
plt.ylabel("Accuracy")
plt.title(f"{ds.upper()} – bert + {llm_type}")
plt.legend()
plt.tight_layout()
plt.show()

ValueError: attempt to get argmax of an empty sequence

## 다음 단계 (논문용 확장 TODO)

이 노트북은 기본적인 표/그림 생성 골격만 포함하고 있습니다. 논문용으로는 다음을 추가로 확장하면 좋습니다.

1. **실험 커버리지 확대**
   - `datasets` / `llm_type` 리스트를 확장하여
     - Qwen2-0.5B / 7B
     - SNLI / MNLI (entailment)
     조합에 대해서도 동일한 요약 테이블과 곡선을 생성.

2. **선택 신호 vs 최종 성능 상관 분석**
   - selection 로그에서 PCL(키워드 신호, corr 신호)과 ILM head effect, probe accuracy를 불러와
   - 레이어별로 최종 acc와의 피어슨/스피어만 상관계수 계산 및 산점도 시각화.

3. **auto vs last / best 비교 그림**
   - 각 태스크·모델에 대해 `delta_sel_last`, `delta_sel_best`를 막대그래프로 표현.
   - 태스크별로 레이어 민감도가 어떻게 다른지 논의에 활용.

4. **본문/부록용 그림 정제**
   - 레이블/폰트 크기, 색상 팔레트 등을 논문 스타일에 맞게 조정.
   - 선택된 대표 태스크(SST-2, CoLA, MNLI, tweet 계열)에 대한 곡선/히트맵을 개별 그림으로 저장.

이 노트북을 기반으로 위 항목들을 채워 넣으면, 논문에 필요한 정량 표와 시각화를 대부분 커버할 수 있습니다.